In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/CA4/*.py .
# !cp -r /content/drive/MyDrive/CA4/conf .

In [ ]:
!pip install hydra-core --no-binary :all:
!pip install wandb

In [ ]:
%load_ext autoreload
%autoreload 2
from train import run
import os
import os.path as osp
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

In [ ]:
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg)
    data_root = cfg.data.data_root
    if not osp.exists(data_root):
        os.makedirs(data_root, exist_ok=True)
        data_root_father = osp.dirname(data_root)
        !wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip -O {data_root_father}/data.zip
        !unzip {data_root_father}/data.zip -d {data_root_father}

In [ ]:
# wandb
import wandb
wandb.login()


# Define a sweep

In [ ]:
sweep_configuration = {
    "name": "sweep-hyperparams",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "epoch_val_dice_score"},
    "parameters": {
        "learning_rate": {
            "values": [1e-4, 1e-5],

        },
        "batch_size": {"values": [8, 16, 32]},
        "epochs": {"values": [30, 120, 300]},
        "img_size": {
            "values": [64, 128],
        },
    },
}
# sweep_id = wandb.sweep(sweep=sweep_configuration, project="pedestrian-detection")
sweep_id = "bp8sw90v"
print(sweep_id)

# Sweep run

In [ ]:
from train import run
def sweep_train():
    from hydra import initialize, compose
    wandb.init()
    with initialize(version_base=None, config_path="conf"):
        cfg = compose(config_name="config",
                overrides=[f"train.optimizer.lr={wandb.config.learning_rate}",
                      f"train.batch_size={wandb.config.batch_size}",
                      f"train.num_epochs={wandb.config.epochs}",
                      f"train.img_size={wandb.config.img_size}",
                      f"wandb.name={wandb.run.name}",])


        run(cfg)

In [ ]:
wandb.agent(sweep_id, function=sweep_train, project="pedestrian-detection")